In [1]:
""" This is the main script for eap ml project, which plays with all ml methods, applicable for cpcrsp analysis
Created on Sat Feb 12 15:19:54 2022
"""

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import dill, time, gc, random, os, warnings
import statsmodels.api as sm
from sklearn.svm import SVR


from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict, GridSearchCV
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNetCV
from sklearn import svm
from sklearn.decomposition import PCA, KernelPCA
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.impute import SimpleImputer
from sklearn.metrics import r2_score

from xgboost import XGBRegressor

import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense, SimpleRNN, LSTM, Activation, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping

plt.style.use('seaborn-white')
warnings.simplefilter(action='ignore')
pd.set_option('display.max_columns', 40)
gc.enable()

path = '../input/mleap-data/'

tempx = pd.read_csv(path + 's_train_X.csv')
tempy = pd.read_csv(path + 's_train_y.csv')
tempx.drop(columns='Unnamed: 0', inplace=True)
tempy.drop(columns='Unnamed: 0', inplace=True)
s_train_X = np.array(tempx)
s_train_y = np.array(tempy)
print(s_train_X.shape,s_train_y.shape)

(1257579, 27) (1257579, 1)


In [15]:
time0 = time.time()
parameters = {'learning_rate': [0.01, 0.02, 0.03, 0.05], 'max_depth':[2,4,6,8,10], 'colsample_bytree':[0.6,0.8], 'subsample':[0.6,0.8],
             'n_estimators':[300, 500, 700, 1000]}

grid_search = GridSearchCV(estimator=XGBRegressor(tree_method='gpu_hist', gpu_id=0, nthread=4, seed=42), 
                           param_grid=parameters, n_jobs =-1, cv = 4, verbose=True)

grid_search.fit(s_train_X, s_train_y)
print(grid_search.best_score_, grid_search.best_params_)

print(r2_score(s_train_y, grid_search.predict(s_train_X)))
print('runtime is ', time.time()-time0)

Fitting 4 folds for each of 320 candidates, totalling 1280 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:  3.0min
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed: 53.4min
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed: 122.2min
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed: 241.7min
[Parallel(n_jobs=-1)]: Done 1246 tasks      | elapsed: 359.1min
[Parallel(n_jobs=-1)]: Done 1280 out of 1280 | elapsed: 389.3min finished


0.012197167300071171 {'colsample_bytree': 0.8, 'learning_rate': 0.01, 'max_depth': 10, 'n_estimators': 1000, 'subsample': 0.8}
0.17187976620294942
runtime is  23458.075414419174


In [14]:
bt_ = XGBRegressor(n_estimators=1000, max_depth=10, eta=0.01, subsample=0.8, colsample_bytree=0.8, tree_method='gpu_hist', gpu_id=0)
bt_.fit(s_train_X, s_train_y)

#[r2_score(s_train_y, bt_.predict(s_train_X))]

tempx = pd.read_csv(path + 's_test_X.csv')
tempy = pd.read_csv(path + 's_test_y.csv')
tempx.drop(columns='Unnamed: 0', inplace=True)
s_test_X = np.array(tempx)
s_test_y = np.array(tempy)
print(s_test_X.shape,s_test_y.shape)

print([r2_score(s_train_y, bt_.predict(s_train_X)), r2_score(s_test_y, bt_.predict(s_test_X))])

(100000, 27) (100000, 1)
[0.08787465979575648, 0.012908522776634235]
